In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
!apt install unzip
!unzip /content/gdrive/MyDrive/Dataset-20230812T070125Z-001.zip -d /content/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Archive:  /content/gdrive/MyDrive/Dataset-20230812T070125Z-001.zip
replace /content/Dataset/test_images/test_images/2.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
import pandas as pd

# Load the dataset
fashion_df = pd.read_csv('/content/Dataset/fashion.csv')

# Preview the dataset
print(fashion_df.head())

   ProductId Gender Category SubCategory ProductType Colour   Usage  \
0      42419  Girls  Apparel     Topwear        Tops  White  Casual   
1      34009  Girls  Apparel     Topwear        Tops  Black  Casual   
2      40143  Girls  Apparel     Topwear        Tops   Blue  Casual   
3      23623  Girls  Apparel     Topwear        Tops   Pink  Casual   
4      47154  Girls  Apparel  Bottomwear      Capris  Black  Casual   

                                  ProductTitle      Image  \
0           Gini and Jony Girls Knit White Top  42419.jpg   
1                Gini and Jony Girls Black Top  34009.jpg   
2  Gini and Jony Girls Pretty Blossom Blue Top  40143.jpg   
3   Doodle Kids Girls Pink I love Shopping Top  23623.jpg   
4             Gini and Jony Girls Black Capris  47154.jpg   

                                            ImageURL  
0  http://assets.myntassets.com/v1/images/style/p...  
1  http://assets.myntassets.com/v1/images/style/p...  
2  http://assets.myntassets.com/v1/images

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import pandas as pd
import numpy as np

# Load dataset
df = fashion_df
df['description'] = df['Gender'] + ' ' + df['Category'] + ' ' + df['SubCategory'] + ' ' + df['ProductType'] + ' ' + df['Colour'] + ' ' + df['Usage'] + ' ' + df['ProductTitle']

# Image preprocessing
def preprocess_image(image_filename):
    image_path = '/content/Dataset/images/' + image_filename
    image = load_img(image_path, target_size=(299, 299))
    image = img_to_array(image)
    image = preprocess_input(image)
    return image.reshape((1, 299, 299, 3))

inception_model = InceptionV3(include_top=False, pooling='avg')
image_features = np.vstack([inception_model.predict(preprocess_image(img_filename)) for img_filename in df['Image']])



1/1 [==============================] - 0s 25ms/step


In [5]:
from tensorflow.keras import backend as K
K.clear_session()


In [8]:
# Text preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['description'])
vocab_size = len(tokenizer.word_index) + 1
descriptions = tokenizer.texts_to_sequences(df['description'])
max_length = max([len(desc) for desc in descriptions])
input_descriptions = [desc[:-1] for desc in descriptions]
input_descriptions = pad_sequences(input_descriptions, maxlen=max_length-1)
target_descriptions = [desc[-1] for desc in descriptions]
target_descriptions = to_categorical(target_descriptions, num_classes=vocab_size)

# Model
image_input = Input(shape=(2048,))
text_input = Input(shape=(max_length-1,))
image_dense = Dense(256, activation='relu')(image_input)
text_embedding = Embedding(vocab_size, 256, input_length=max_length-1)(text_input)
text_lstm = LSTM(256)(text_embedding)
merged = Concatenate()([image_dense, text_lstm])
output = Dense(vocab_size, activation='softmax')(merged)

model = Model(inputs=[image_input, text_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training
model.fit([image_features, input_descriptions], target_descriptions, epochs=10, batch_size=32) # Reduced batch size

Epoch 1/10
182/182 [==============================] - 11s 40ms/step - loss: 1.5628
Epoch 2/10
182/182 [==============================] - 2s 11ms/step - loss: 0.5917
Epoch 3/10
182/182 [==============================] - 2s 12ms/step - loss: 0.4050
Epoch 4/10
182/182 [==============================] - 1s 8ms/step - loss: 0.2949
Epoch 5/10
182/182 [==============================] - 1s 7ms/step - loss: 0.2147
Epoch 6/10
182/182 [==============================] - 1s 6ms/step - loss: 0.1717
Epoch 7/10
182/182 [==============================] - 1s 6ms/step - loss: 0.1307
Epoch 8/10
182/182 [==============================] - 1s 7ms/step - loss: 0.0970
Epoch 9/10
182/182 [==============================] - 1s 7ms/step - loss: 0.0931
Epoch 10/10
182/182 [==============================] - 1s 7ms/step - loss: 0.0824


In [11]:
# Image preprocessing
def preprocess_image(image_path):
    image = load_img(image_path, target_size=(299, 299))
    image = img_to_array(image)
    image = preprocess_input(image)
    return image.reshape((1, 299, 299, 3))

# Example usage
image_path = '/content/Dataset/images/10037.jpg'
description = generate_description(model, image_path)
print(description)


1/1 [==============================] - 0s 472ms/step
shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes shoes


In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, AdditiveAttention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.applications import ResNet50
# Load dataset
# Ensure that df['description'] contains the correct text data
# df['Image'] should contain the paths to the images

# Text preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['description'])
vocab_size = len(tokenizer.word_index) + 1
descriptions = tokenizer.texts_to_sequences(df['description'])
max_length = max([len(desc) for desc in descriptions]) # Including target word

# Input descriptions (all words except the last)
input_descriptions = [desc[:-1] for desc in descriptions]
input_descriptions = pad_sequences(input_descriptions, maxlen=max_length-1)

# Target descriptions (last word of each sequence)
target_descriptions = [desc[-1] for desc in descriptions]
target_descriptions = to_categorical(target_descriptions, num_classes=vocab_size)

# Image preprocessing
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50

def preprocess_image(image_filename):
    image_path = '/content/Dataset/images/' + image_filename
    image = load_img(image_path, target_size=(224, 224)) # Change target size to (224, 224)
    image = img_to_array(image)
    image = preprocess_input(image)
    return image.reshape((1, 224, 224, 3))

resnet_model = ResNet50(include_top=False, pooling='avg')
image_features = np.vstack([resnet_model.predict(preprocess_image(img_filename)) for img_filename in df['Image']])

# Prediction function will require modification according to the new architecture


1/1 [==============================] - 0s 22ms/step


In [22]:
# Text preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['description'])
vocab_size = len(tokenizer.word_index) + 1
descriptions = tokenizer.texts_to_sequences(df['description'])
max_length = max([len(desc) for desc in descriptions])

# Input descriptions (all words except the last)
input_descriptions = [desc[:-1] for desc in descriptions]
input_descriptions = pad_sequences(input_descriptions, maxlen=max_length-1)

# Prepare the target descriptions
target_descriptions = [desc[1:] for desc in descriptions] # Shift by one position
target_descriptions = pad_sequences(target_descriptions, maxlen=max_length-1)
target_descriptions = [to_categorical(desc, num_classes=vocab_size) for desc in target_descriptions]
target_descriptions = np.array(target_descriptions)

In [39]:
from tensorflow.keras.layers import RepeatVector

# Encoder
image_input = Input(shape=(2048,))
image_features_layer = Dense(256, activation='relu')(image_input)
image_features_repeat = RepeatVector(max_length-1)(image_features_layer)

# Decoder
text_input = Input(shape=(max_length-1,))
text_embedding = Embedding(vocab_size, 256, mask_zero=True)(text_input)
text_lstm = LSTM(256, return_sequences=True)(text_embedding)

# Concatenate image features and text
decoder_input = Concatenate(axis=-1)([image_features_repeat, text_lstm])
output_lstm = LSTM(256, return_sequences=True)(decoder_input)
output = TimeDistributed(Dense(vocab_size, activation='softmax'))(output_lstm)

model = Model([image_input, text_input], output)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [40]:
model.fit([image_features, input_descriptions], target_descriptions, epochs=20, batch_size=64)


Epoch 1/20
46/46 [==============================] - 20s 236ms/step - loss: 4.5295
Epoch 2/20
46/46 [==============================] - 7s 157ms/step - loss: 3.4720
Epoch 3/20
46/46 [==============================] - 7s 152ms/step - loss: 3.0367
Epoch 4/20
46/46 [==============================] - 7s 146ms/step - loss: 2.6726
Epoch 5/20
46/46 [==============================] - 7s 143ms/step - loss: 2.3530
Epoch 6/20
46/46 [==============================] - 5s 119ms/step - loss: 2.0643
Epoch 7/20
46/46 [==============================] - 8s 163ms/step - loss: 1.7975
Epoch 8/20
46/46 [==============================] - 6s 123ms/step - loss: 1.5758
Epoch 9/20
46/46 [==============================] - 7s 159ms/step - loss: 1.4034
Epoch 10/20
46/46 [==============================] - 5s 117ms/step - loss: 1.2741
Epoch 11/20
46/46 [==============================] - 7s 163ms/step - loss: 1.1593
Epoch 12/20
46/46 [==============================] - 5s 118ms/step - loss: 1.0656
Epoch 13/20
46/46 [=====

In [41]:
def preprocess_image1(image_path):
    image = load_img(image_path, target_size=(299, 299)) # Adjust target_size if using ResNet50
    image = img_to_array(image)
    image = preprocess_input(image)
    return image.reshape((1, 299, 299, 3))

In [47]:
def generate_description(model, image_path):
    # Preprocess the image
    image = preprocess_image1(image_path)
    image_features = resnet_model.predict(image)

    # Start token
    start_token = [1]
    input_sequence = pad_sequences([start_token], maxlen=max_length-1)

    # Generate the description iteratively
    description = []
    for i in range(max_length - 1):
        predictions = model.predict([image_features, input_sequence], verbose=0)
        predicted_word_index = np.argmax(predictions[0, i, :])

        # Translate the predicted word index to a word
        word = tokenizer.index_word.get(predicted_word_index, ' ')
        if word == ' ':
            break

        description.append(word)

        # Update the input sequence for the next iteration
        input_sequence = np.append(input_sequence, [[predicted_word_index]], axis=1)
        input_sequence = pad_sequences(input_sequence, maxlen=max_length-1)

    return ' '.join(description)

# Example usage
image_path = '/content/Dataset/test_images/test_images/3.jpg'
description = generate_description(model, image_path)
print(description)


1/1 [==============================] - 0s 24ms/step
apparel apparel apparel apparel apparel apparel apparel apparel apparel shoes shoes navy casual nike benetton navy girls black


In [45]:
model.save("image_caption_model.h5")


In [49]:
# Save tokenizer
import pickle

with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, AdditiveAttention, Concatenate, TimeDistributed
from tensorflow.keras.models import Model

image_input = Input(shape=(2048,))
image_features_layer = Dense(256, activation='relu')(image_input)
text_input = Input(shape=(max_length-1,))
text_embedding = Embedding(vocab_size, 256, mask_zero=True)(text_input)
text_lstm = LSTM(256, return_sequences=True)(text_embedding)
attention = AdditiveAttention()([text_lstm, image_features_layer])
decoder_input = Concatenate(axis=-1)([text_lstm, attention])
output = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder_input) # Apply Dense to each time step
model = Model([image_input, text_input], output)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training
model.fit([image_features, input_descriptions], target_descriptions, epochs=10, batch_size=32)


Epoch 1/10
91/91 [==============================] - 19s 144ms/step - loss: 3.9510
Epoch 2/10
91/91 [==============================] - 6s 68ms/step - loss: 2.3932
Epoch 3/10
91/91 [==============================] - 8s 89ms/step - loss: 1.8982
Epoch 4/10
91/91 [==============================] - 6s 65ms/step - loss: 1.6529
Epoch 5/10
91/91 [==============================] - 8s 84ms/step - loss: 1.4875
Epoch 6/10
91/91 [==============================] - 6s 61ms/step - loss: 1.3727
Epoch 7/10
91/91 [==============================] - 8s 84ms/step - loss: 1.2780
Epoch 8/10
91/91 [==============================] - 6s 65ms/step - loss: 1.1958
Epoch 9/10
91/91 [==============================] - 8s 83ms/step - loss: 1.1227
Epoch 10/10
91/91 [==============================] - 6s 65ms/step - loss: 1.0591
